In [1]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens1MReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data("/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/data") #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

DataSplitter_Holdout: Verifying data consistency...
DataSplitter_Holdout: Verifying data consistency... Passed!
DataSplitter_Holdout: DataReader: Movielens1M
	Num items: 3883
	Num users: 6040
	Train 		quota 80.00 (80.00), 	interactions 800167, 	density 3.41E-02
	Validation 	quota 10.00 (10.00), 	interactions 100021, 	density 4.26E-03
	Test 		quota 10.00 (10.00), 	interactions 100021, 	density 4.26E-03



	ICM name: ICM_genres, Num features: 18, feature occurrences: 6408, density 9.17E-02
	ICM name: ICM_year, Num features: 1, feature occurrences: 3883, density 1.00E+00
	UCM name: UCM_all, Num features: 3469, feature occurrences: 24160, density 1.15E-03


DataSplitter_Holdout: Done.
EvaluatorHoldout: Ignoring 79 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 64 ( 1.1%) Users that have less than 1 test interactions


In [2]:
import optuna
import numpy as np
import pandas as pd
import os

def create_objective_d(recommender_instance, URM_train, URM_validation, hyperparam_space, evaluator_validation, cutoff, directory_path):
    def objective(trial):
        # Gather and set hyperparameters
        hyperparams = {name: trial.suggest_categorical(name, space) for name, space in hyperparam_space.items()}

        # Initialize and train the recommender
        
        recommender_instance.fit(**hyperparams)

        # Evaluate recommender
        user_id_array = np.arange(URM_validation.shape[0])
        
        result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
        result_df['hyperparams'] = str(hyperparams)

        # Assuming df_row is the DataFrame containing the row to append

        filename = directory_path + recommender_instance.RECOMMENDER_NAME + ".csv"

        # Check if file exists
        if os.path.isfile(filename):
            # If it exists, append without writing the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
        else:
            # If it doesn't exist, create it, write the header
           pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)
    
        return result_df.loc[cutoff]['NDCG']

    return objective


In [4]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

from Diffusion.DiffusionRecommender import DiffusionAutoencoderRecommender_OptimizerMask
import torch



In [11]:
itemKNN_instance2 = ItemKNNCFRecommender(URM_train=URM_train)
itemKNN_instance2.fit(topK=96, shrink =59, similarity ='cosine', normalize=True, feature_weighting='TF-IDF')
result_df, _ = evaluator_validation.evaluateRecommender(itemKNN_instance2)
result_df

ItemKNNCFRecommender: URM Detected 211 ( 5.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 3883 (100.0%), 3990.75 column/sec. Elapsed time 0.97 sec
EvaluatorHoldout: Processed 5961 (100.0%) in 3.91 sec. Users per second: 1523


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.172068,0.225724,0.152794,0.090517,0.114132,0.392586,0.1897,0.16186,0.744674,0.584341,...,0.986921,0.734934,0.986921,0.064196,8.296197,0.995206,0.183438,0.76718,1.686127,0.066146
50,0.101315,0.394449,0.388508,0.034652,0.112369,0.40243,0.268621,0.160718,0.925851,0.729563,...,0.986921,0.913742,0.986921,0.115281,9.205504,0.998123,0.32941,0.851267,1.367718,0.342828


In [5]:
#load validation(best_model) models instances

itemKNN_instance = ItemKNNCFRecommender(URM_train=URM_train)
itemKNN_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/ItemKNNCFRecommender_cosine_best_model")

slim_elastic_instance = SLIMElasticNetRecommender(URM_train=URM_train)
slim_elastic_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/SLIMElasticNetRecommender_best_model")

#matrix_fact_instance = PureSVDRecommender(URM_train=URM_train)
#matrix_fact_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/PureSVDRecommender_best_model")

matrix_fact_instance = IALSRecommender(URM_train=URM_train)
matrix_fact_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/IALSRecommender_best_model")


auto_diffusion_instance = DiffusionAutoencoderRecommender_OptimizerMask(URM_train=URM_train, use_gpu=False)
auto_diffusion_instance.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/DiffusionAutoencoderRecommender_best_model")


ItemKNNCFRecommender: URM Detected 211 ( 5.4%) items with no interactions.
ItemKNNCFRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/ItemKNNCFRecommender_cosine_best_model'
ItemKNNCFRecommender: Loading complete
SLIMElasticNetRecommender: URM Detected 211 ( 5.4%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/SLIMElasticNetRecommender_best_model'
SLIMElasticNetRecommender: Loading complete
IALSRecommender: URM Detected 211 ( 5.4%) items with no interactions.
IALSRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/IALSRecommender_best_model'
IALSRecommender: Loading complete
DiffusionAutoencoderRecommender: URM Detected 211 ( 5.4%) items with no interactions.
DiffusionAutoencoderRecommender: Loading model from 

In [7]:

itemKNN_instance_test = ItemKNNCFRecommender(URM_train=URM_train+URM_validation)
itemKNN_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/ItemKNNCFRecommender_cosine_best_model_last")

slim_elastic_instance_test = SLIMElasticNetRecommender(URM_train=URM_train)
slim_elastic_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/SLIMElasticNetRecommender_best_model_last")

pure_svd_instance_test = PureSVDRecommender(URM_train=URM_train)
pure_svd_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/PureSVDRecommender_best_model_last")


matrix_fact_instance_test = IALSRecommender(URM_train=URM_train)
matrix_fact_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/IALSRecommender_best_model_last")


auto_diffusion_instance_test = DiffusionAutoencoderRecommender_OptimizerMask(URM_train=URM_train, use_gpu=False)
auto_diffusion_instance_test.load_model(folder_path="/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models", file_name="/DiffusionAutoencoderRecommender_best_model_last")


ItemKNNCFRecommender: URM Detected 194 ( 5.0%) items with no interactions.
ItemKNNCFRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/ItemKNNCFRecommender_cosine_best_model_last'
ItemKNNCFRecommender: Loading complete
SLIMElasticNetRecommender: URM Detected 211 ( 5.4%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/SLIMElasticNetRecommender_best_model_last'
SLIMElasticNetRecommender: Loading complete
PureSVDRecommender: URM Detected 211 ( 5.4%) items with no interactions.
PureSVDRecommender: Loading model from file '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/teams/Movielens1M/models/PureSVDRecommender_best_model_last'
PureSVDRecommender: Loading complete
IALSRecommender: URM Detected 211 ( 5.4%) items with no interactions.
IALSRecommender: Loading model from file 

In [11]:
result_df, _ = evaluator_validation.evaluateRecommender(matrix_fact_instance)
result_df

EvaluatorHoldout: Processed 5961 (100.0%) in 3.62 sec. Users per second: 1648


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.186345,0.245822,0.169608,0.09839,0.124377,0.414177,0.20549,0.177583,0.780909,0.626442,...,0.986921,0.770695,0.986921,0.073846,8.557885,0.996611,0.211011,0.79138,1.595006,0.066525
50,0.111149,0.434059,0.427869,0.03877,0.127246,0.423121,0.293169,0.176459,0.93944,0.786603,...,0.986921,0.927152,0.986921,0.138295,9.492902,0.998854,0.39517,0.877844,1.238788,0.347542


In [15]:
from datetime import datetime
from RecommenderLuca.TwoLevelRec import TwoLevelRec

inference_timesteps = {1,5,25,50}
recommenders_instances = {itemKNN_instance, slim_elastic_instance, matrix_fact_instance}
print(recommenders_instance for recommenders_instance in recommenders_instances)

today_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

for cutoff in cutoff_list:
    for traditional_rec in recommenders_instances:
        for inference_timestep in inference_timesteps:
             # Create the directory with today's date
            directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
            os.makedirs(directory_path, exist_ok=True)
            auto_diffusion_instance.inference_timesteps = inference_timestep
            print(traditional_rec.RECOMMENDER_NAME)
            two_level_rec1 = TwoLevelRec(URM_train=URM_train, recommender1= auto_diffusion_instance, recommender2=traditional_rec)
            print(two_level_rec1.RECOMMENDER_NAME)

            # Define the hyperparameter space
            itemknn_hyperparam_space = {
                "cutoff": list(range(cutoff, 300+cutoff)),
            }

            # Create an objective function
            objective = create_objective_d(recommender_instance=two_level_rec1, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=itemknn_hyperparam_space, evaluator_validation=evaluator_validation, cutoff = cutoff)

            # Create the study and run optimization
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=50,show_progress_bar=True)
            
            two_level_rec1.fit(** study.best_params)
            filename = "best_models" + str(inference_timestep) + "_timesteps" + ".csv"
            filename = os.path.join(directory_path, filename)
            filename2 = auto_diffusion_instance.RECOMMENDER_NAME + "_" + traditional_rec.RECOMMENDER_NAME + "_" + str(inference_timestep)
            filename2 = os.path.join(directory_path, filename2)
            result_df, _ = evaluator_validation.evaluateRecommender(two_level_rec1)
            result_df["RECOMMENDER"] = two_level_rec1.RECOMMENDER_NAME
            result_df["PARAMS"] = str(study.best_params)

            
            # Check if file exists
            if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
            else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        

[I 2023-07-28 08:07:39,031] A new study created in memory with name: no-name-b231a0c4-3694-400b-837c-d9685ab1065e
  0%|          | 0/50 [00:00<?, ?it/s]

<generator object <genexpr> at 0x7fc0a5e26b30>
SLIMElasticNetRecommender
Recommender_Base_Class: URM Detected 211 ( 5.4%) items with no interactions.
DiffusionAutoencoderRecommender_SLIMElasticNetRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5961 (100.0%) in 4.37 sec. Users per second: 1363


Best trial: 0. Best value: 0.224937:   2%|▏         | 1/50 [00:06<04:59,  6.10s/it]

[I 2023-07-28 08:07:45,173] Trial 0 finished with value: 0.22493688850671822 and parameters: {'cutoff': 46}. Best is trial 0 with value: 0.22493688850671822.
EvaluatorHoldout: Processed 5961 (100.0%) in 4.40 sec. Users per second: 1354


Best trial: 1. Best value: 0.225064:   4%|▍         | 2/50 [00:12<04:49,  6.03s/it]

...

[I 2023-07-28 10:41:43,201] A new study created in memory with name: no-name-ea7d4bc0-4743-4d27-9cb8-4b7efc0d0fe8
  0%|          | 0/50 [00:00<?, ?it/s]

PureSVDRecommender
Recommender_Base_Class: URM Detected 211 ( 5.4%) items with no interactions.
DiffusionAutoencoderRecommender_PureSVDRecommender_TwoLevelRec


  0%|          | 0/50 [00:01<?, ?it/s]


[W 2023-07-28 10:41:44,959] Trial 0 failed with parameters: {'cutoff': 82} because of the following error: TypeError("object of type 'NoneType' has no len()").
Traceback (most recent call last):
  File "/Users/lucaortolomo/opt/anaconda3/envs/tesi/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_96897/1651610602.py", line 18, in objective
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
  File "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Evaluation/Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Evaluation/Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, scores_batch = recommender_obj

TypeError: object of type 'NoneType' has no len()

In [ ]:
inference_timesteps = {1,5,25,50}
recommenders_instances = {itemKNN_instance, slim_elastic_instance, matrix_fact_instance}

today_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

for cutoff in cutoff_list:

    for traditional_rec in recommenders_instances:
        for inference_timestep in inference_timesteps:
            # Create the directory with today's date
            directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
            os.makedirs(directory_path, exist_ok=True)
            auto_diffusion_instance.inference_timesteps = inference_timestep
            two_level_rec1 = TwoLevelRec(URM_train=URM_train, recommender1= traditional_rec, recommender2=auto_diffusion_instance)
            print(two_level_rec1.RECOMMENDER_NAME)
            # Define the hyperparameter space
            two_levels_hyperparam_space = {
                "cutoff": list(range(cutoff, 300 + cutoff)),
            }

            # Create an objective function
            objective = create_objective_d(recommender_instance=two_level_rec1, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_levels_hyperparam_space, evaluator_validation=evaluator_validation)

            # Create the study and run optimization
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=50,show_progress_bar=True)
            
            filename = "best_models" + str(inference_timestep) + "_timesteps" + ".csv"
            filename = os.path.join(directory_path, filename)
            result_df, _ = evaluator_validation.evaluateRecommender(two_level_rec1)
            result_df["RECOMMENDER"] = two_level_rec1.RECOMMENDER_NAME
            result_df["PARAMS"] = str(study.best_params)

            
            # Check if file exists
            if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
            else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

    

    

In [11]:
from RecommenderLuca import TwoLevelRec
import importlib

# Make some changes to your_module.py

importlib.reload(TwoLevelRec)
from RecommenderLuca.TwoLevelRec import TwoLevelRec

In [14]:
from datetime import datetime
from RecommenderLuca.TwoLevelRec import TwoLevelRec

inference_timesteps = [1,5,25,50]
recommenders_instances = [itemKNN_instance, slim_elastic_instance, matrix_fact_instance]
recommenders_instances_test = [itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test]

print(recommenders_instance for recommenders_instance in recommenders_instances)

today_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


for inference_timestep in inference_timesteps:
    auto_diffusion_instance.inference_timesteps = inference_timestep
    auto_diffusion_instance_test.inference_timesteps = inference_timestep
    two_level_rec = TwoLevelRec(URM_train=URM_train, recommender1= auto_diffusion_instance, recommender2=itemKNN_instance, max_cutoff=max(cutoff_list))
    for cutoff in cutoff_list:
        # Define the hyperparameter space
        two_level_rec_hyperparam_space = {
            "n_items_to_rank": list(range(cutoff, 300+cutoff)),
        }
        for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
            two_level_rec.set_rec2(traditional_rec)
             # Create the directory with today's date
            directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
            os.makedirs(directory_path, exist_ok=True)
            
            print(traditional_rec.RECOMMENDER_NAME)
            
            print(two_level_rec.RECOMMENDER_NAME)

           

            # Create an objective function
            objective = create_objective_d(recommender_instance=two_level_rec, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_level_rec_hyperparam_space, evaluator_validation=evaluator_validation, cutoff = cutoff, directory_path=directory_path)

            # Create the study and run optimization
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=50,show_progress_bar=True)
            
            two_level_rec1 = TwoLevelRec(URM_train=URM_train+URM_validation, recommender1= auto_diffusion_instance_test, recommender2=traditional_rec_test, max_cutoff=max(cutoff_list))
            two_level_rec1.fit(** study.best_params)
            filename = "best_models_last_" + str(inference_timestep) + "_timesteps" + ".csv"
            filename = os.path.join(directory_path, filename)
            filename2 = auto_diffusion_instance.RECOMMENDER_NAME + "_" + traditional_rec.RECOMMENDER_NAME + "_" + str(inference_timestep)
            filename2 = os.path.join(directory_path, filename2)
            result_df, _ = evaluator_test.evaluateRecommender(two_level_rec1)
            result_df["RECOMMENDER"] = two_level_rec1.RECOMMENDER_NAME
            result_df["PARAMS"] = str(study.best_params)

            
            # Check if file exists
            if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
            else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        

<generator object <genexpr> at 0x7f78ccf02dd0>
Recommender_Base_Class: URM Detected 211 ( 5.4%) items with no interactions.


[I 2023-07-31 03:41:55,253] A new study created in memory with name: no-name-a76fb45c-a600-4c94-ab41-eb1149f47109
  0%|          | 0/50 [00:00<?, ?it/s]

SLIMElasticNetRecommender
DiffusionAutoencoderRecommender_SLIMElasticNetRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5961 (100.0%) in 4.28 sec. Users per second: 1391


Best trial: 0. Best value: 0.224704:   2%|▏         | 1/50 [00:04<03:31,  4.31s/it]

[I 2023-07-31 03:41:59,563] Trial 0 finished with value: 0.22470394825909362 and parameters: {'n_items_to_rank': 260}. Best is trial 0 with value: 0.22470394825909362.
EvaluatorHoldout: Processed 5961 (100.0%) in 4.16 sec. Users per second: 1431


...

Best trial: 0. Best value: 0.240484:  98%|█████████▊| 49/50 [03:03<00:03,  3.77s/it]

[I 2023-07-31 05:18:40,956] Trial 48 finished with value: 0.24048367348205787 and parameters: {'n_items_to_rank': 286}. Best is trial 0 with value: 0.24048367348205787.
EvaluatorHoldout: Processed 5961 (100.0%) in 3.75 sec. Users per second: 1589


Best trial: 0. Best value: 0.240484: 100%|██████████| 50/50 [03:07<00:00,  3.75s/it]

[I 2023-07-31 05:18:44,739] Trial 49 finished with value: 0.24048367348205787 and parameters: {'n_items_to_rank': 340}. Best is trial 0 with value: 0.24048367348205787.
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.


EvaluatorHoldout: Processed 5976 (100.0%) in 3.94 sec. Users per second: 1516


In [17]:
from datetime import datetime
from RecommenderLuca.TwoLevelRec import TwoLevelRec

inference_timesteps = [1,5,25,50]
recommenders_instances = [itemKNN_instance, slim_elastic_instance, matrix_fact_instance]
recommenders_instances_test = [itemKNN_instance_test, slim_elastic_instance_test, matrix_fact_instance_test]

print(recommenders_instance for recommenders_instance in recommenders_instances)

today_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')


for inference_timestep in inference_timesteps:
    auto_diffusion_instance.inference_timesteps = inference_timestep
    auto_diffusion_instance_test.inference_timesteps = inference_timestep

    for cutoff in cutoff_list:
        # Define the hyperparameter space
        two_level_rec_hyperparam_space = {
            "n_items_to_rank": list(range(cutoff, 300+cutoff)),
        }
        for traditional_rec,traditional_rec_test in zip(recommenders_instances, recommenders_instances_test):
            two_level_rec = TwoLevelRec(URM_train=URM_train, recommender1= traditional_rec, recommender2=auto_diffusion_instance, max_cutoff=max(cutoff_list))
             # Create the directory with today's date
            directory_path = os.path.join('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/experiment_data', today_date,'cutoff_'+str(cutoff), str(inference_timestep))
            os.makedirs(directory_path, exist_ok=True)
            
            print(traditional_rec.RECOMMENDER_NAME)
            
            print(two_level_rec.RECOMMENDER_NAME)

           

            # Create an objective function
            objective = create_objective_d(recommender_instance=two_level_rec, URM_train= URM_train, URM_validation=URM_validation, hyperparam_space=two_level_rec_hyperparam_space, evaluator_validation=evaluator_validation, cutoff = cutoff, directory_path=directory_path)

            # Create the study and run optimization
            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=50,show_progress_bar=True)
            
            two_level_rec_test = TwoLevelRec(URM_train=URM_train+URM_validation, recommender1= traditional_rec_test, recommender2=auto_diffusion_instance_test, max_cutoff=max(cutoff_list))
            two_level_rec_test.fit(** study.best_params)
            filename = "best_models_last_" + str(inference_timestep) + "_timesteps" + ".csv"
            filename = os.path.join(directory_path, filename)
            filename2 = auto_diffusion_instance.RECOMMENDER_NAME + "_" + traditional_rec.RECOMMENDER_NAME + "_" + str(inference_timestep)
            filename2 = os.path.join(directory_path, filename2)
            result_df, _ = evaluator_test.evaluateRecommender(two_level_rec_test)
            result_df["RECOMMENDER"] = two_level_rec_test.RECOMMENDER_NAME
            result_df["PARAMS"] = str(study.best_params)
            print(two_level_rec.RECOMMENDER_NAME)
            print(two_level_rec_test.RECOMMENDER_NAME)


            
            # Check if file exists
            if os.path.isfile(filename):
                # If it exists, append without writing the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='a', header=False, index=False)
            else:
                # If it doesn't exist, create it, write the header
                pd.DataFrame(result_df.loc[10]).transpose().to_csv(filename, mode='w', header=True, index=False)

        

<generator object <genexpr> at 0x7f78b2dc9970>
Recommender_Base_Class: URM Detected 211 ( 5.4%) items with no interactions.


[I 2023-07-31 09:13:52,292] A new study created in memory with name: no-name-be35ab91-3aea-4088-b41e-17db8af4566e
  0%|          | 0/50 [00:00<?, ?it/s]

ItemKNNCFRecommender
ItemKNNCFRecommender_DiffusionAutoencoderRecommender_TwoLevelRec
EvaluatorHoldout: Processed 5961 (100.0%) in 5.00 sec. Users per second: 1192


Best trial: 0. Best value: 0.214609:   2%|▏         | 1/50 [00:05<04:05,  5.02s/it]

[I 2023-07-31 09:13:57,311] Trial 0 finished with value: 0.21460894126138869 and parameters: {'n_items_to_rank': 94}. Best is trial 0 with value: 0.21460894126138869.
EvaluatorHoldout: Processed 5961 (100.0%) in 4.99 sec. Users per second: 1194


...

Best trial: 2. Best value: 0.255849:  98%|█████████▊| 49/50 [37:50<00:46, 46.37s/it]

[I 2023-07-31 19:39:45,966] Trial 48 finished with value: 0.25430162799666206 and parameters: {'n_items_to_rank': 292}. Best is trial 2 with value: 0.25584911961398826.
EvaluatorHoldout: Processed 5961 (100.0%) in 46.27 sec. Users per second: 129


Best trial: 2. Best value: 0.255849: 100%|██████████| 50/50 [38:37<00:00, 46.34s/it]

[I 2023-07-31 19:40:32,274] Trial 49 finished with value: 0.2547224864281644 and parameters: {'n_items_to_rank': 65}. Best is trial 2 with value: 0.25584911961398826.
Recommender_Base_Class: URM Detected 194 ( 5.0%) items with no interactions.


EvaluatorHoldout: Processed 5976 (100.0%) in 46.39 sec. Users per second: 129
IALSRecommender_DiffusionAutoencoderRecommender_TwoLevelRec
IALSRecommender_DiffusionAutoencoderRecommender_TwoLevelRec
